In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt 
import seaborn as sns 

from sklearn.model_selection import train_test_split , GridSearchCV 
from sklearn.metrics import classification_report 
from sklearn.preprocessing import LabelEncoder , StandardScaler 

from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import RandomForestClassifier , GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB 

from xgboost import XGBClassifier 
from sklearn.neighbors import KNeighborsClassifier

import warnings 
warnings.filterwarnings("ignore")



import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



/kaggle/input/playground-series-s4e2/sample_submission.csv
/kaggle/input/playground-series-s4e2/train.csv
/kaggle/input/playground-series-s4e2/test.csv


In [2]:
train= pd.read_csv("/kaggle/input/playground-series-s4e2/train.csv")
test = pd.read_csv("/kaggle/input/playground-series-s4e2/test.csv")


In [3]:
train.sample(10)

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
4711,4711,Female,18.000000,1.630000,58.000000,no,yes,3.000000,3.000000,Sometimes,no,2.000000,no,1.000000,1.000000,Sometimes,Public_Transportation,Normal_Weight
10582,10582,Female,25.919241,1.611452,102.093223,yes,yes,3.000000,3.000000,Sometimes,no,1.023328,no,0.050930,1.000000,Sometimes,Public_Transportation,Obesity_Type_III
3924,3924,Male,20.206358,1.789555,137.767787,yes,yes,2.499626,3.000000,Sometimes,no,2.868167,no,1.847802,0.785701,Sometimes,Public_Transportation,Obesity_Type_II
3076,3076,Male,18.000000,1.620938,68.250249,yes,no,2.000000,1.402771,Sometimes,no,1.919629,no,0.000000,1.000000,no,Public_Transportation,Overweight_Level_II
507,507,Female,25.919241,1.669701,104.572712,yes,yes,3.000000,3.000000,Sometimes,no,1.353167,no,0.095517,0.543960,Sometimes,Public_Transportation,Obesity_Type_III
17056,17056,Female,23.000000,1.700000,50.000000,yes,yes,3.000000,4.000000,Sometimes,no,2.000000,no,2.000000,1.000000,no,Public_Transportation,Insufficient_Weight
3430,3430,Female,25.000000,1.640000,51.000000,no,yes,3.000000,1.000000,Sometimes,no,2.000000,no,2.000000,1.000000,Sometimes,Public_Transportation,Normal_Weight
18846,18846,Female,21.682367,1.694952,133.764730,yes,yes,3.000000,3.000000,Sometimes,no,1.338241,no,1.488090,0.950438,Sometimes,Public_Transportation,Obesity_Type_III
8008,8008,Male,19.000000,1.800000,68.000000,no,yes,2.000000,3.000000,Sometimes,no,2.000000,no,1.000000,1.000000,Sometimes,Public_Transportation,Normal_Weight
9071,9071,Female,25.991886,1.626179,105.037203,yes,yes,3.000000,3.000000,Sometimes,no,3.000000,no,0.008013,0.447224,Sometimes,Public_Transportation,Obesity_Type_III


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20758 entries, 0 to 20757
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              20758 non-null  int64  
 1   Gender                          20758 non-null  object 
 2   Age                             20758 non-null  float64
 3   Height                          20758 non-null  float64
 4   Weight                          20758 non-null  float64
 5   family_history_with_overweight  20758 non-null  object 
 6   FAVC                            20758 non-null  object 
 7   FCVC                            20758 non-null  float64
 8   NCP                             20758 non-null  float64
 9   CAEC                            20758 non-null  object 
 10  SMOKE                           20758 non-null  object 
 11  CH2O                            20758 non-null  float64
 12  SCC                             

In [5]:
train.columns

Index(['id', 'Gender', 'Age', 'Height', 'Weight',
       'family_history_with_overweight', 'FAVC', 'FCVC', 'NCP', 'CAEC',
       'SMOKE', 'CH2O', 'SCC', 'FAF', 'TUE', 'CALC', 'MTRANS', 'NObeyesdad'],
      dtype='object')

In [6]:
train.isnull().sum()

id                                0
Gender                            0
Age                               0
Height                            0
Weight                            0
family_history_with_overweight    0
FAVC                              0
FCVC                              0
NCP                               0
CAEC                              0
SMOKE                             0
CH2O                              0
SCC                               0
FAF                               0
TUE                               0
CALC                              0
MTRANS                            0
NObeyesdad                        0
dtype: int64

In [7]:
train.duplicated().sum()

0

In [8]:
train.describe()

,id,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE
count,20758.00000,20758.000000,20758.000000,20758.000000,20758.000000,20758.000000,20758.000000,20758.000000,20758.000000
mean,10378.50000,23.841804,1.700245,87.887768,2.445908,2.761332,2.029418,0.981747,0.616756
std,5992.46278,5.688072,0.087312,26.379443,0.533218,0.705375,0.608467,0.838302,0.602113
min,0.00000,14.000000,1.450000,39.000000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,5189.25000,20.000000,1.631856,66.000000,2.000000,3.000000,1.792022,0.008013,0.000000
50%,10378.50000,22.815416,1.700000,84.064875,2.393837,3.000000,2.000000,1.000000,0.573887
75%,15567.75000,26.000000,1.762887,111.600553,3.000000,3.000000,2.549617,1.587406,1.000000
max,20757.00000,61.000000,1.975663,165.057269,3.000000,4.000000,3.000000,3.000000,2.000000


# transform 

In [9]:
lab = LabelEncoder()

In [10]:
tobj = train.select_dtypes(include="object")
tint = train.select_dtypes(exclude="object")

In [11]:
for i in range(0,tobj.shape[1]):
 tobj.iloc[:,i]=lab.fit_transform(tobj.iloc[:,i])


In [12]:
tobj = tobj.astype("int")

In [13]:
data =pd.concat([tobj,tint] ,axis=1)

In [14]:
test = pd.read_csv("/kaggle/input/playground-series-s4e2/test.csv")



In [15]:
tobj = test.select_dtypes(include="object")
tint = test.select_dtypes(exclude="object")
for i in range(0,tobj.shape[1]):
    tobj.iloc[:,i]=lab.fit_transform(tobj.iloc[:,i])
tobj = tobj.astype("int")
test= pd.concat([tobj,tint] ,axis=1)

In [16]:
data

,Gender,family_history_with_overweight,FAVC,CAEC,SMOKE,SCC,CALC,MTRANS,NObeyesdad,id,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE
0,1,1,1,2,0,0,1,3,6,0,24.443011,1.699998,81.669950,2.000000,2.983297,2.763573,0.000000,0.976473
1,0,1,1,1,0,0,2,0,1,1,18.000000,1.560000,57.000000,2.000000,3.000000,2.000000,1.000000,1.000000
2,0,1,1,2,0,0,2,3,0,2,18.000000,1.711460,50.165754,1.880534,1.411685,1.910378,0.866045,1.673584
3,0,1,1,2,0,0,1,3,4,3,20.952737,1.710730,131.274851,3.000000,3.000000,1.674061,1.467863,0.780199
4,1,1,1,2,0,0,1,3,6,4,31.641081,1.914186,93.798055,2.679664,1.971472,1.979848,1.967973,0.931721
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20753,1,1,1,2,0,0,1,3,3,20753,25.137087,1.766626,114.187096,2.919584,3.000000,2.151809,1.330519,0.196680
20754,1,0,1,1,0,0,1,3,0,20754,18.000000,1.710000,50.000000,3.000000,4.000000,1.000000,2.000000,1.000000
20755,1,1,1,2,0,0,2,3,3,20755,20.101026,1.819557,105.580491,2.407817,3.000000,2.000000,1.158040,1.198439
20756,1,1,1,2,0,0,2,0,6,20756,33.852953,1.700000,83.520113,2.671238,1.971472,2.144838,0.000000,0.973834


In [17]:
sc = StandardScaler()
scal1=data[['Age']]
scal2=data[['Weight']] 
data[['Age']] = sc.fit_transform(scal1)
data[['Weight']] = sc.fit_transform(scal2)
data

,Gender,family_history_with_overweight,FAVC,CAEC,SMOKE,SCC,CALC,MTRANS,NObeyesdad,id,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE
0,1,1,1,2,0,0,1,3,6,0,0.105699,1.699998,-0.235713,2.000000,2.983297,2.763573,0.000000,0.976473
1,0,1,1,1,0,0,2,0,1,1,-1.027052,1.560000,-1.170931,2.000000,3.000000,2.000000,1.000000,1.000000
2,0,1,1,2,0,0,2,3,0,2,-1.027052,1.711460,-1.430012,1.880534,1.411685,1.910378,0.866045,1.673584
3,0,1,1,2,0,0,1,3,4,3,-0.507929,1.710730,1.644770,3.000000,3.000000,1.674061,1.467863,0.780199
4,1,1,1,2,0,0,1,3,6,4,1.371197,1.914186,0.224054,2.679664,1.971472,1.979848,1.967973,0.931721
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20753,1,1,1,2,0,0,1,3,3,20753,0.227725,1.766626,0.996987,2.919584,3.000000,2.151809,1.330519,0.196680
20754,1,0,1,1,0,0,1,3,0,20754,-1.027052,1.710000,-1.436296,3.000000,4.000000,1.000000,2.000000,1.000000
20755,1,1,1,2,0,0,2,3,3,20755,-0.657669,1.819557,0.670717,2.407817,3.000000,2.000000,1.158040,1.198439
20756,1,1,1,2,0,0,2,0,6,20756,1.760067,1.700000,-0.165574,2.671238,1.971472,2.144838,0.000000,0.973834


In [18]:
sc = StandardScaler()
scal1=test[['Age']]
scal2=test[['Weight']] 
test[['Age']] = sc.fit_transform(scal1)
test[['Weight']] = sc.fit_transform(scal2)
test

,Gender,family_history_with_overweight,FAVC,CAEC,SMOKE,SCC,CALC,MTRANS,id,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE
0,1,1,1,2,0,0,2,3,20758,0.508163,1.848294,1.273786,2.938616,3.000000,2.825629,0.855400,0.000000
1,0,1,1,2,0,0,2,3,20759,-0.509128,1.600000,-0.818988,2.000000,1.000000,3.000000,1.000000,0.000000
2,0,1,1,2,0,0,2,3,20760,0.353000,1.643355,0.927432,3.000000,3.000000,2.621877,0.000000,0.250502
3,1,1,1,2,0,0,2,3,20761,-0.512705,1.553127,0.623672,2.000000,2.977909,2.786417,0.094851,0.000000
4,0,1,1,2,0,0,2,3,20762,0.353000,1.627396,0.668336,3.000000,3.000000,2.653531,0.000000,0.741069
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13835,1,1,0,2,0,0,2,3,34593,-0.107749,1.721384,-0.358246,2.813234,3.000000,1.000000,0.807076,0.778632
13836,0,0,1,2,0,0,2,3,34594,0.870277,1.590000,-0.972181,3.000000,3.000000,2.000000,0.000000,0.000000
13837,0,0,1,1,0,0,2,3,34595,-0.175379,1.585547,-1.647125,3.000000,2.273740,2.000000,1.949840,1.000000
13838,1,1,1,2,0,0,3,3,34596,-0.509128,1.620000,-1.316865,2.000000,3.000000,2.000000,3.000000,2.000000


# Model

In [19]:
x=data.drop(['NObeyesdad','id'] ,axis=1)
y=data['NObeyesdad']
x

,Gender,family_history_with_overweight,FAVC,CAEC,SMOKE,SCC,CALC,MTRANS,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE
0,1,1,1,2,0,0,1,3,0.105699,1.699998,-0.235713,2.000000,2.983297,2.763573,0.000000,0.976473
1,0,1,1,1,0,0,2,0,-1.027052,1.560000,-1.170931,2.000000,3.000000,2.000000,1.000000,1.000000
2,0,1,1,2,0,0,2,3,-1.027052,1.711460,-1.430012,1.880534,1.411685,1.910378,0.866045,1.673584
3,0,1,1,2,0,0,1,3,-0.507929,1.710730,1.644770,3.000000,3.000000,1.674061,1.467863,0.780199
4,1,1,1,2,0,0,1,3,1.371197,1.914186,0.224054,2.679664,1.971472,1.979848,1.967973,0.931721
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20753,1,1,1,2,0,0,1,3,0.227725,1.766626,0.996987,2.919584,3.000000,2.151809,1.330519,0.196680
20754,1,0,1,1,0,0,1,3,-1.027052,1.710000,-1.436296,3.000000,4.000000,1.000000,2.000000,1.000000
20755,1,1,1,2,0,0,2,3,-0.657669,1.819557,0.670717,2.407817,3.000000,2.000000,1.158040,1.198439
20756,1,1,1,2,0,0,2,0,1.760067,1.700000,-0.165574,2.671238,1.971472,2.144838,0.000000,0.973834


In [20]:
x_train,x_test,y_train,y_test = train_test_split(x,y,train_size=0.2 , random_state=10) 

In [21]:
model1= LogisticRegression()
model2= RandomForestClassifier()
model3= GaussianNB()
model4= KNeighborsClassifier()
model5 = XGBClassifier(learning_rate=0.1, max_depth=3, n_estimators=300)
model6= GradientBoostingClassifier()

In [22]:
def models(model):
    model.fit(x_train,y_train)
    pre = model.predict(x_test)
    print(classification_report(pre,y_test))

In [23]:
models(model1)

              precision    recall  f1-score   support

           0       0.89      0.78      0.83      2268
           1       0.66      0.73      0.69      2243
           2       0.77      0.77      0.77      2324
           3       0.96      0.92      0.94      2751
           4       1.00      0.99      0.99      3252
           5       0.57      0.64      0.60      1714
           6       0.62      0.61      0.62      2055

    accuracy                           0.80     16607
   macro avg       0.78      0.78      0.78     16607
weighted avg       0.81      0.80      0.80     16607



In [24]:
models(model2)

              precision    recall  f1-score   support

           0       0.93      0.92      0.92      2036
           1       0.86      0.83      0.84      2556
           2       0.85      0.88      0.87      2256
           3       0.96      0.96      0.96      2668
           4       1.00      1.00      1.00      3224
           5       0.73      0.77      0.75      1819
           6       0.79      0.77      0.78      2048

    accuracy                           0.89     16607
   macro avg       0.87      0.88      0.87     16607
weighted avg       0.89      0.89      0.89     16607



In [25]:
models(model3)

              precision    recall  f1-score   support

           0       0.86      0.70      0.77      2442
           1       0.46      0.63      0.53      1832
           2       0.41      0.46      0.43      2089
           3       0.98      0.52      0.67      5040
           4       1.00      0.96      0.98      3355
           5       0.28      0.56      0.37       975
           6       0.23      0.53      0.32       874

    accuracy                           0.64     16607
   macro avg       0.60      0.62      0.58     16607
weighted avg       0.76      0.64      0.67     16607



In [26]:
models(model4)

              precision    recall  f1-score   support

           0       0.85      0.70      0.77      2437
           1       0.56      0.59      0.57      2317
           2       0.74      0.67      0.70      2544
           3       0.94      0.91      0.93      2769
           4       1.00      0.99      1.00      3233
           5       0.42      0.53      0.47      1539
           6       0.52      0.59      0.55      1768

    accuracy                           0.75     16607
   macro avg       0.72      0.71      0.71     16607
weighted avg       0.76      0.75      0.75     16607



In [27]:
models(model5)

              precision    recall  f1-score   support

           0       0.94      0.92      0.93      2053
           1       0.86      0.86      0.86      2462
           2       0.88      0.88      0.88      2320
           3       0.96      0.97      0.96      2634
           4       1.00      1.00      1.00      3226
           5       0.77      0.78      0.78      1915
           6       0.80      0.80      0.80      1997

    accuracy                           0.90     16607
   macro avg       0.89      0.89      0.89     16607
weighted avg       0.90      0.90      0.90     16607



In [28]:
models(model6)

              precision    recall  f1-score   support

           0       0.94      0.91      0.92      2053
           1       0.85      0.86      0.85      2446
           2       0.88      0.88      0.88      2320
           3       0.96      0.97      0.96      2631
           4       1.00      1.00      1.00      3226
           5       0.76      0.78      0.77      1890
           6       0.81      0.79      0.80      2041

    accuracy                           0.89     16607
   macro avg       0.88      0.88      0.88     16607
weighted avg       0.89      0.89      0.89     16607



In [29]:
grid={'n_estimators' : [10,200,300] ,
      'learning_rate' : [0.1,0.01,0.001] ,
        'max_depth' : [3,5,8]
     }
scorer="accuracy"

In [30]:
#model55 = GridSearchCV(model5 , grid, scoring=scorer , n_jobs= -1)
#model55.fit(x_train,y_train)
#print(model55.best_params_)
#print(model55.best_score_)



In [31]:
#model66 = GridSearchCV(model6 , grid, scoring=scorer , n_jobs= -1)
#model66.fit(x_train,y_train)
#print(model66.best_params_)
#print(model66.best_score_)


In [32]:
testx = test.drop('id' , axis=1)
preex = model5.predict(testx)

In [33]:
subb = pd.DataFrame({"id":test['id'] ,
              "NObeyesdad" : preex})
subb

,id,NObeyesdad
0,20758,3
1,20759,5
2,20760,4
3,20761,2
4,20762,4
...,...,...
13835,34593,6
13836,34594,1
13837,34595,0
13838,34596,1


In [34]:
# Refit the LabelEncoder with all possible labels
lab = LabelEncoder()
lab.fit(train['NObeyesdad'])  # or fit with all your labels

# Now inverse transform should work
subb = pd.DataFrame({
    "id": test['id'],
    "NObeyesdad": lab.inverse_transform(preex)
})


In [35]:
subb['NObeyesdad'] = lab.inverse_transform(preex)
subb

,id,NObeyesdad
0,20758,Obesity_Type_II
1,20759,Overweight_Level_I
2,20760,Obesity_Type_III
3,20761,Obesity_Type_I
4,20762,Obesity_Type_III
...,...,...
13835,34593,Overweight_Level_II
13836,34594,Normal_Weight
13837,34595,Insufficient_Weight
13838,34596,Normal_Weight


In [36]:
subb.to_csv("submission.csv" , index=False)